In [4]:
import os
import requests
from bs4 import BeautifulSoup
import zipfile
import pandas as pd

# URL base da Receita Federal
base_url = "https://arquivos.receitafederal.gov.br/cnpj/dados_abertos_cnpj/"

# Função para listar diretórios disponíveis na URL base
def listar_diretorios_base():
    response = requests.get(base_url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        links = soup.find_all('a')
        diretorios = [link.get('href') for link in links if link.get('href') and link.get('href').startswith('20')]
        return diretorios
    else:
        print(f"Erro ao acessar a URL: {response.status_code}")
        return []

# Função para baixar e renomear arquivos ZIP de um diretório específico
def baixar_arquivo_zip(diretorio, nome_arquivo_original, nome_arquivo_local):
    url = f"{base_url}{diretorio}{nome_arquivo_original}"
    response = requests.get(url)
    if response.status_code == 200:
        # Diretório local para salvar os arquivos
        local_dir = r"C:\Users\Kkk\3D Objects\TrustWork"
        
        # Criar diretório local, se não existir
        if not os.path.exists(local_dir):
            os.makedirs(local_dir)
        
        # Caminho completo do arquivo local
        caminho_arquivo = os.path.join(local_dir, nome_arquivo_local)
        
        # Verificar se o arquivo já existe
        if not os.path.exists(caminho_arquivo):
            # Salvar o arquivo
            with open(caminho_arquivo, 'wb') as file:
                file.write(response.content)
            print(f"Arquivo {nome_arquivo_local} baixado com sucesso no diretório {local_dir}!")
        else:
            print(f"O arquivo {nome_arquivo_local} já existe no diretório {local_dir}. Download não realizado.")
    else:
        print(f"Erro ao acessar a URL: {response.status_code}")

# Função para extrair arquivos ZIP
def extrair_arquivo_zip(diretorio, nome_arquivo_local):
    local_dir = r"C:\Users\Kkk\3D Objects\TrustWork"
    caminho_arquivo = os.path.join(local_dir, nome_arquivo_local)
    extrair_para = os.path.join(local_dir, diretorio.strip('/'))
    
    # Verificar se o arquivo já foi extraído
    if os.path.exists(extrair_para):
        print(f"O diretório {extrair_para} já existe. Extraindo novamente.")
        
    # Extrair o arquivo ZIP
    with zipfile.ZipFile(caminho_arquivo, 'r') as zip_ref:
        zip_ref.extractall(extrair_para)
    print(f"Arquivo {nome_arquivo_local} extraído com sucesso para o diretório {extrair_para}!")

# Listar diretórios e baixar arquivos desejados
diretorios = listar_diretorios_base()
diretorios_desejados = ["2024-12/", "2025-01/"]

for diretorio in diretorios_desejados:
    if diretorio in diretorios:
        nome_arquivo_empresas = f"Empresas1-{diretorio.strip('/')}.zip"
        nome_arquivo_estabelecimentos = f"Estabelecimentos1-{diretorio.strip('/')}.zip"
        
        baixar_arquivo_zip(diretorio, "Empresas1.zip", nome_arquivo_empresas)
        extrair_arquivo_zip(diretorio, nome_arquivo_empresas)
        
        baixar_arquivo_zip(diretorio, "Estabelecimentos1.zip", nome_arquivo_estabelecimentos)
        extrair_arquivo_zip(diretorio, nome_arquivo_estabelecimentos)
    else:
        print(f"Diretório {diretorio} não encontrado.")

O arquivo Empresas1-2024-12.zip já existe no diretório C:\Users\Kkk\3D Objects\TrustWork. Download não realizado.
O diretório C:\Users\Kkk\3D Objects\TrustWork\2024-12 já existe. Extraindo novamente.
Arquivo Empresas1-2024-12.zip extraído com sucesso para o diretório C:\Users\Kkk\3D Objects\TrustWork\2024-12!
O arquivo Estabelecimentos1-2024-12.zip já existe no diretório C:\Users\Kkk\3D Objects\TrustWork. Download não realizado.
O diretório C:\Users\Kkk\3D Objects\TrustWork\2024-12 já existe. Extraindo novamente.
Arquivo Estabelecimentos1-2024-12.zip extraído com sucesso para o diretório C:\Users\Kkk\3D Objects\TrustWork\2024-12!
O arquivo Empresas1-2025-01.zip já existe no diretório C:\Users\Kkk\3D Objects\TrustWork. Download não realizado.
O diretório C:\Users\Kkk\3D Objects\TrustWork\2025-01 já existe. Extraindo novamente.
Arquivo Empresas1-2025-01.zip extraído com sucesso para o diretório C:\Users\Kkk\3D Objects\TrustWork\2025-01!
O arquivo Estabelecimentos1-2025-01.zip já existe n

In [ ]:
import os
import pandas as pd

# Diretório base onde os arquivos foram extraídos
local_dir = r"C:\Users\Kkk\3D Objects\TrustWork"

# Função para adicionar cabeçalho e criar DataFrames com nomes específicos
def processar_arquivos(diretorio):
    caminho_diretorio = os.path.join(local_dir, diretorio)
    
    # Verificar se o diretório existe
    if os.path.exists(caminho_diretorio):
        # Listar arquivos no diretório
        arquivos = os.listdir(caminho_diretorio)
        
        # Identificar e processar arquivos "EMPRE" e "ESTA"
        for arquivo in arquivos:
            if "EMPRE" in arquivo:
                caminho_arquivo = os.path.join(caminho_diretorio, arquivo)
                colunas_empre = ["cnpj_em", "razao", "nat_jur", "qual_resp", "cap_soc", "port_emp"]
                df_empre = pd.read_csv(caminho_arquivo, delimiter=';', header=None, names=colunas_empre, encoding='latin1', engine='python')
                # Nomear o DataFrame
                globals()[f"df_empre_{diretorio.replace('-', '_')}"] = df_empre
                print(f"Primeiras linhas do DataFrame df_empre_{diretorio.replace('-', '_')}:\n", df_empre.head(), "\n")
            elif "ESTA" in arquivo:
                caminho_arquivo = os.path.join(caminho_diretorio, arquivo)
                colunas_esta = ["cnpj_es", "cnpj_2", "cnpj_3", "matriz_filial", "fantasia", "sit_cad", "data_sit_cad",
                                "mot_sit_cad", "nome_cid_ext", "pais", "dat_ini_at", "cnae", "cnae_2", "tip_log",
                                "logradouro", "numero", "complemento", "bairro", "cep", "uf", "municipio", "ddd", "tel1",
                                "ddd2", "tel2", "dddfax", "fax", "email", "sit_esp", "dat_sit_exp"]
                df_esta = pd.read_csv(caminho_arquivo, delimiter=';', header=None, names=colunas_esta, encoding='latin1', engine='python')
                # Nomear o DataFrame
                globals()[f"df_esta_{diretorio.replace('-', '_')}"] = df_esta
                print(f"Primeiras linhas do DataFrame df_esta_{diretorio.replace('-', '_')}:\n", df_esta.head(), "\n")

# Listar diretórios a serem processados
diretorios_a_processar = ["2024-12", "2025-01"]

# Processar arquivos em cada diretório
for diretorio in diretorios_a_processar:
    processar_arquivos(diretorio)

Primeiras linhas do DataFrame df_empre_2024_12:
                                              cnpj_em  razao  nat_jur  \
0                                 BANCO DO BRASIL SA   2038       10   
1  ASSOCIACAO DE AMIGOS DE BAIRRO DO CONJ PAULISTANO   3999       16   
2      WM&R EMPREITEIRA DE CONSTRUCAO CIVIL LIMITADA   2240       49   
3  CASA CARIDADE LUZETE ROBERTA DE MORAIS CONJ PA...   3999       16   
4  ASSOCIACAO DO PEQUENO ADOLECENTE DO CONJ PAULI...   3999       16   

         qual_resp  cap_soc port_emp  
0  120000000000,00      5.0      NaN  
1             0,00      5.0      NaN  
2             0,00      5.0      NaN  
3             0,00      5.0      NaN  
4             0,00      5.0      NaN   



In [ ]:
df_empre.head()